## Code for performing integration by Gaussian quadrature 

Adapted from *Computational Physics*, by Mark Newman.

The code estimates the $m^{th}$ zero of the Legendre polynomial $P_n(\cos \theta)$ using the expression 

$\theta_m^{(n)} = \dfrac{4m - 1}{4n + 2} \pi + \dfrac{1}{8n^2} \cot \left( \dfrac{4m - 1}{4n + 2}\right) + O(n^{-3}) \hspace{30pt}$ Eq. 22.16.6 of *Handbook of Mathematical Functions* by Abramowitz & Stegun.

It also makes use of the following recurrence relation (ibid, Eq. 22.7.10):

$(n + 1) P_{n+1}(\cos \theta) = (2n + 1) P_n(\cos \theta) \cos \theta - n P_{n-1}(\cos \theta).$

In [1]:
def gaussQuad(func, N, a=1, b=1):

    '''Uses Gaussian quadrature to compute the integral of function 
    "func" over the range (a, b) using N sample points.'''
    
    from numpy import ones,copy,cos,tan,pi,linspace

    # Initial approximation to roots of the Legendre polynomial
    arr = linspace(3, 4*N - 1, N)/(4*N + 2)
    x = cos(pi * arr + 1/(8 * N * N * tan(arr)))

    # Find roots using Newton's method
    epsilon = 1e-15
    delta = 1.0
    while delta > epsilon:
        p0 = ones(N, float)
        p1 = copy(x)
        
        for k in range(1,N):
            p0, p1 = p1, ((2*k + 1)*x*p1 - k*p0)/(k + 1)
            
        dp = (N + 1)*(p0 - x*p1)/(1 - x*x)
        dx = p1/dp
        x = x - dx
        delta = max(abs(dx))

    # Calculate the weights (for x-range (-1,1))
    w = 2*(N + 1)*(N + 1)/(N*N*(1 - x*x)*dp*dp)
    
    # Scale to range (a,b)
    xs = 0.5*(b - a)*x + 0.5*(b + a)
    ws = 0.5*(b - a)*w
    
    # Perform the integration
    integ = 0.0
    for k in range(N):
        integ = integ + ws[k] * func(xs[k])

    print("The integral with %d sample points is %.12f" %(N, integ))
    return integ